<a href="https://colab.research.google.com/github/diwadu/dw_matrix_road_sign/blob/master/dw_matrix_3_d_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import os
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
  y_train.shape, y_test.shape

((34799,), (4410,))

In [0]:
#dodajemy wymiar (robimy macierz dla tensorflowa)
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
y_train.shape, y_test.shape

((34799, 43), (4410, 43))

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
model = Sequential([

  Conv2D(filters=65, kernel_size=(3,3), activation='relu', 
         input_shape=input_shape),
  Flatten(),
  Dense(num_classes, activation='softmax')
])

#model.summary()
#model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
#model.fit(X_train, y_train)

In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
      Conv2D(filters=65, kernel_size=(3,3), activation='relu', 
         input_shape=input_shape),
      Flatten(),
      Dense(num_classes, activation='softmax')
  ])


def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', 
              optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose= params_fit.get('verbose'),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return logdir

In [0]:
model = get_cnn_v1(input_shape, num_classes)
logdir = train_model(model, X_train, y_train)

In [0]:
%tensorboard --logdir logs

In [0]:
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      MaxPool2D(),
      Dropout(0.5),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),
      
      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.5),

      Dense(num_classes, activation='softmax')
  ])

In [0]:
model = get_cnn_v2(input_shape, num_classes)
params_fit = {
    'epochs': 25
}
train_model(model, X_train, y_train)

'logs/20200401-161637'

In [0]:
%tensorboard --logdir logs